# Greeks Coberture for options
* The objetive of this proyect is make a coberture 

Using the finance option **AMAZON**


In [1]:
## Import Packages
import pandas as pd
import numpy as np

**MAKING an data analysis**

In [2]:
## Load data 
data= pd.read_excel('Proyecto Griegas .xlsx')

In [3]:
data.describe()

,Strike,Last Price,Bid,Ask,Change,Implied Volatility,Stock price
count,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000
mean,1535.296222,309.382951,296.780019,303.304555,2.048261,1.112457,100.371214
std,1613.390977,530.210910,509.789755,515.094628,47.149631,2.249404,4.438275
min,45.000000,0.010000,0.000000,0.000000,-272.700000,0.000000,92.130000
25%,161.000000,5.242500,4.375000,4.537500,0.000000,0.000000,98.150000
50%,245.000000,65.600000,67.000000,71.425000,0.000000,0.439700,102.570000
75%,3260.000000,234.000000,181.550000,193.250000,0.000000,0.673125,102.790000
max,5500.000000,3256.610000,3087.500000,3106.500000,673.730000,17.312500,112.910000


**EDA**
> Removing Variables that not need 

* we need only the adj close in Bid y Ask, and stock price 

In [4]:
df=data[['Last Trade Date','Strike','Bid', 'Ask','Stock price']]
df['Adj Close']=df[['Bid','Ask']].mean(axis=1)
df['Last Trade Date']=df['Last Trade Date'].astype(str).str[:10]
df['Last Trade Date']=df['Last Trade Date'].apply(lambda x: pd.to_datetime(x,format='%Y/%m/%d'))
df.set_index('Last Trade Date',inplace=True)

<ipython-input-4-5a4c3c3fe163>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Adj Close']=df[['Bid','Ask']].mean(axis=1)
<ipython-input-4-5a4c3c3fe163>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Trade Date']=df['Last Trade Date'].astype(str).str[:10]
<ipython-input-4-5a4c3c3fe163>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

## Definiremos nuestra r basandonos en la TIE 
* Tomando datos historicos del año pasado **por BANXICO**


In [5]:
data_TIE= pd.read_excel('Proyecto Griegas .xlsx','TIE',)
data_TIE=data_TIE.iloc[16:,:2]
data_TIE.columns=['Fecha','TIE']
TIE=round(data_TIE['TIE'].mean(),2)
print('La TIE promedio en el ultimo año es:',TIE,'%')

La TIE promedio en el ultimo año es: 9.35 %


## Definimos los parametros a trabajar
* Necesitamos los datos historicos de 1 año atras sobre la accion

**Objetivo:**
> * Obtener mu
>* Obtener Sigma

In [6]:
##Chargin data
Historical_Data= pd.read_excel('Proyecto Griegas .xlsx','Parámetros')
Historical_Data=Historical_Data.iloc[2:,4:6].copy()

In [7]:
##trating the data
Historical_Data.columns=('Fecha','St')
Historical_Data['St']=Historical_Data['St'].astype(float)
##Log return
Historical_Data['ln(St)']=np.log(Historical_Data['St'].astype(float))
Historical_Data[['St','ln(St)']]=Historical_Data[['St','ln(St)']].round(2)
Historical_Data['Fecha']=Historical_Data['Fecha'].apply(lambda x: pd.to_datetime(x,format='%Y/%m/%d'))
Historical_Data.set_index('Fecha',inplace=True)

In [8]:
Historical_Data.sort_index(ascending=False,inplace=True)

In [9]:
Historical_Data['%_Change']=round(Historical_Data['St'].pct_change(),5)
Historical_Data['Log_Return']=round(np.log(Historical_Data.St) - np.log(Historical_Data.St.shift(1)),5)

In [10]:
Historical_Data.dropna(inplace=True)
Historical_Data.sort_index(ascending=True,inplace=True)
Historical_Data.head()

,St,ln(St),%_Change,Log_Return
Fecha,,,,
2022-05-09,108.79,4.69,-0.00064,-0.00064
2022-05-10,108.86,4.69,0.03312,0.03258
2022-05-11,105.37,4.66,-0.01459,-0.01470
2022-05-12,106.93,4.67,-0.05422,-0.05574
2022-05-13,113.06,4.73,0.02031,0.02010


* Encontrando los parametros

In [11]:
Tau,mu,sigma,r,K=[1,
                  round(np.mean(Historical_Data['ln(St)']),2),
                  round(np.var(Historical_Data['ln(St)']),2),
                  round(TIE,2),
                  90]
print('Nuestros Parámetros son:\n''Tau:',Tau,'\nmu:',mu,'\nsigma:',sigma,'\nr:',r,'\nK:',K)

Nuestros Parámetros son:
Tau: 1 
mu: 4.67 
sigma: 0.02 
r: 9.35 
K: 90
